In [3]:
from palmerpenguins import load_penguins
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from imblearn import FunctionSampler
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df = load_penguins()
# Find unbalanced class
Counter(df["species"])
# Count missing values
df.isnull().sum(axis=1)
#OnehotEncode categorical values
categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])
cate = categorical_transformer.fit(df)
new_df = cate.transform(df)
# print(new_df)
# Outlier values
upper_limit = df["bill_length_mm"].mean() + 3 * df["bill_length_mm"].std()
lower_limit = df["bill_length_mm"].mean() - 3 * df["bill_length_mm"].std()
print(upper_limit, lower_limit)

60.300680966341 27.543178682781807


In [15]:
r = df.isnull().sum(axis=1)
np.array(r)

array([0, 0, 0, 5, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Extract the data and target from the DataFrame

In [39]:
X = df.loc[:, 'island':]
y = df['species']
a = X.isnull().sum(axis=1)
b = list(a[a/5 < 0.1].index)
X.drop(b)

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
3,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
8,Torgersen,34.1,18.1,193.0,3475.0,NaN,2007
9,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007
10,Torgersen,37.8,17.1,186.0,3300.0,NaN,2007
11,Torgersen,37.8,17.3,180.0,3700.0,NaN,2007
47,Dream,37.5,18.9,179.0,2975.0,NaN,2007
178,Biscoe,44.5,14.3,216.0,4100.0,NaN,2007
218,Biscoe,46.2,14.4,214.0,4650.0,NaN,2008
256,Biscoe,47.3,13.8,216.0,4725.0,NaN,2009
268,Biscoe,44.5,15.7,217.0,4875.0,NaN,2009


In [26]:
class MyCustomTransformer(BaseEstimator):
    def __init__(self, hp):
        self.hp = hp
        self.row_null = None
    def fit(self, X, y=None):
        try:
            num_of_null = X.isnull().sum(axis=1)
            self.row_null = list(num_of_null[num_of_null/len(X.columns) >= 0.1].index)
        except Exception as e:
            print(e)
        finally:
            return self

    def transform(self, X, y=None):
        try:
            X.drop(self.row_null)
        except Exception as e:
            print(e)
        finally:
            return X